<a href="https://colab.research.google.com/github/XG11/ECE420FinalProject/blob/main/ECE420FinalProjectPythonPrototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter, gaussian_gradient_magnitude
from scipy.spatial.distance import euclidean
from scipy.signal import convolve2d
import matplotlib.pyplot as plt

In [ ]:
def BlurredImage(image, sigma):
  gaussian = np.zeros((image.shape[0], image.shape[1]))
  for i in range(0, image.shape[0]):
    for j in range(0, image.shape[1]):
      gaussian[i][j] = 1/(2 * np.pi * sigma * sigma) * np.exp(-1 * (np.square(i) + np.square(j))/(2 * np.square(sigma)))
  L = convolve2d(image, gaussian, mode='same')
  return L

def octaves(image, sigma, scale, octave):
  l_images = []
  num_octave = []
  im_copy = image
  for o in range(0, octave):
    if(o != 0):
      im_copy = cv2.resize(im_copy, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
    r_image = im_copy.copy()
    for s in range(0, scale):
       r_image = BlurredImage(r_image, sigma)
       num_octave.append(r_image)
    l_images.append(num_octave)
    for s in range(0, scale):
      num_octave.pop()
  return l_images

def DOG(l_images):
  d_images = []
  num_octave = []
  for i in range(0, len(l_images))
    for j in range(0, len(l_images[i]) - 1):
      difference = l_images[j+1] - l_images[j]
      num_octave.append(difference)
    d_images.append(num_octave)
    for k in range(0, len(l_images[i]) - 1):
      num_octave.pop()
  return d_images




In [ ]:
def keypoints(d_images, threshold = 0.3):
  keypoints = []
  scale_factor = 2
  for o, octave in enumerate(d_images):
    for s, scale in octave:
      current = scale
      prev = scale - 1
      next = scale + 1
      for i in range(1, current.shape[0] - 1):
        for j in range(1, current.shape[1] - 1):
          current_pixel = current[i][j]
          if abs.current_pixel < threshold:
            continue

          neighborhood = [prev[i-1:i+2, j-1:j+2], current[i-1:i+2, j-1:j+2], next[i-1:i+2, j-1:j+2]]
          max_val = np.max(neighborhood)
          min_val = np.min(neighborhood)
          if current_pixel == max_val or current_pixel == min_val:
            original_i = i * scale_factor
            original_j = j * scale_factor
            keypoints.append([o, s, original_i, original_j])

  return keypoints


In [11]:
def M_theta_calculation(image, keypoint, radius=8):
 ''' for k in keypoints:
          M.append(np.sqrt(np.square(l_images[0][k[1]][k[2]+1,k[3]]-l_images[0][k[1]][k[2]-1,k[3]]) + np.square(l_images[0][k[1]][k[2],k[3]+1]-l_images[0][k[1]][k[2],k[3]-1])))
          theta.append(np.arctan2((l_images[0][k[1]][k[2],k[3]+1]-l_images[0][k[1]][k[2],k[3]-1])/(l_images[0][k[1]][k[2]+1,k[3]]-l_images[0][k[1]][k[2]-1,k[3]])))
'''
    x, y = keypoint[2], keypoint[3]
    gradient_magnitudes = []
    gradient_orientations = []

    # Compute gradient magnitudes and orientations around the keypoint
    for i in range(-radius, radius + 1):
        for j in range(-radius, radius + 1):
            if (0 < x + i < image.shape[0] - 1) and (0 < y + j < image.shape[1] - 1):
                dx = image[x + i, y + j + 1] - image[x + i, y + j - 1]
                dy = image[x + i + 1, y + j] - image[x + i - 1, y + j]
                magnitude = np.sqrt(dx**2 + dy**2)
                orientation = np.degrees(np.arctan2(dy, dx)) % 360
                gradient_magnitudes.append(magnitude)
                gradient_orientations.append(orientation)
    return gradient_magnitudes, gradient_orientations

def histogram(gradient_magnitudes, gradient_orientations):
  orientation_histogram = np.zeros(36)
  for magnitude, orientation in zip(gradient_magnitudes, gradient_orientations):
    bin_index = int(orientation // 10)
    orientation_histogram[bin_index] += magnitude
    dominant_orientation = np.argmax(orientation_histogram) * 10